In [1]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.preprocessing import MinMaxScaler

# ===============================
# 1. Baca dan Normalisasi Dataset
# ===============================

# Baca dataset CSV (pastikan kolom: "Cost", "Clicks (destination)", "Impressions")
df = pd.read_csv('../dataset/test-dataset.csv')  # Ganti dengan path file CSV Anda

# Tampilkan beberapa informasi awal
print("Data asli:")
print(df.head())

# Buat scaler untuk normalisasi (min-max scaling)
scaler = MinMaxScaler()

# Kolom yang akan dinormalisasi
cols = ['Cost', 'Clicks (destination)', 'Impressions']
df_norm_values = scaler.fit_transform(df[cols])

# Tambahkan kolom hasil normalisasi ke dataframe
df[['Cost_norm', 'Clicks_norm', 'Impressions_norm']] = df_norm_values

print("\nData setelah normalisasi:")
print(df[['Cost', 'Cost_norm', 'Clicks (destination)', 'Clicks_norm', 'Impressions', 'Impressions_norm']].head())

# ====================================
# 2. Definisikan Fuzzy System dengan Data Normalisasi
# ====================================

# Karena data sudah dinormalisasi, semua variabel input berada dalam rentang [0,1]
cost_norm = ctrl.Antecedent(np.arange(0, 1.001, 0.001), 'cost_norm')
clicks_norm = ctrl.Antecedent(np.arange(0, 1.001, 0.001), 'clicks_norm')
impressions_norm = ctrl.Antecedent(np.arange(0, 1.001, 0.001), 'impressions_norm')

# Output ranking juga pada skala 0 sampai 1
ranking = ctrl.Consequent(np.arange(0, 1.001, 0.001), 'ranking')

# ===============================
# 3. Fungsi Keanggotaan (Membership Functions)
# ===============================

# Karena nilai sudah dinormalisasi, kita ubah parameter berdasarkan rasio nilai asli.

# Misal: untuk Cost, pada data asli:
#   - "low": 0 sampai 1000, di mana 1000/18418.35 ≈ 0.0543.
#   - "medium": 500 sampai 5000, yaitu sekitar [500/18418.35, 5000/18418.35] ~ [0.0272, 0.2716].
#   - "high": mulai dari 1000 hingga max, yaitu [0.0543, 1, 1].
cost_norm['low'] = fuzz.trimf(cost_norm.universe, [0, 0, 0.0543])
cost_norm['medium'] = fuzz.trimf(cost_norm.universe, [0.0272, 0.0543, 0.2716])
cost_norm['high'] = fuzz.trimf(cost_norm.universe, [0.0543, 1, 1])

# Untuk Clicks, pada data asli:
#   - "low": sampai 30000, 30000/548656 ≈ 0.0547.
#   - "medium": dari 15000 ke 300000, yaitu sekitar [15000/548656, 100000/548656, 300000/548656] ~ [0.0273, 0.1823, 0.546].
#   - "high": dari 100000, yaitu [0.1823, 1, 1].
clicks_norm['low'] = fuzz.trimf(clicks_norm.universe, [0, 0, 0.0547])
clicks_norm['medium'] = fuzz.trimf(clicks_norm.universe, [0.0273, 0.1823, 0.546])
clicks_norm['high'] = fuzz.trimf(clicks_norm.universe, [0.1823, 1, 1])

# Untuk Impressions, pada data asli:
#   - "low": sampai 600000, 600000/29406372 ≈ 0.0204.
#   - "medium": dari 600000 ke 3000000, sekitar [0.0204, (1500000/29406372 ≈ 0.051), (3000000/29406372 ≈ 0.102)].
#   - "high": dari 1500000, yaitu [0.051, 1, 1].
impressions_norm['low'] = fuzz.trimf(impressions_norm.universe, [0, 0, 0.0204])
impressions_norm['medium'] = fuzz.trimf(impressions_norm.universe, [0.0204, 0.051, 0.102])
impressions_norm['high'] = fuzz.trimf(impressions_norm.universe, [0.051, 1, 1])

# Untuk Ranking (output) kita tentukan secara relatif:
ranking['low'] = fuzz.trimf(ranking.universe, [0, 0, 0.5])
ranking['medium'] = fuzz.trimf(ranking.universe, [0.25, 0.5, 0.75])
ranking['high'] = fuzz.trimf(ranking.universe, [0.5, 1, 1])

# ====================================
# 4. Buat Aturan Fuzzy (Fuzzy Rules)
# ====================================

# Contoh aturan. Anda bisa menambah atau mengubah sesuai dengan logika bisnis dan data:
# Aturan untuk kombinasi Cost (low), Clicks (low), Impressions (?)
rule1  = ctrl.Rule(cost_norm['low']    & clicks_norm['low']    & impressions_norm['low'],    ranking['low'])    # score: 1+0+0 = 1
rule2  = ctrl.Rule(cost_norm['low']    & clicks_norm['low']    & impressions_norm['medium'], ranking['medium']) # score: 1+0+0.5 = 1.5
rule3  = ctrl.Rule(cost_norm['low']    & clicks_norm['low']    & impressions_norm['high'],   ranking['high'])   # score: 1+0+1 = 2

# Aturan untuk kombinasi Cost (low), Clicks (medium), Impressions (?)
rule4  = ctrl.Rule(cost_norm['low']    & clicks_norm['medium'] & impressions_norm['low'],    ranking['medium']) # score: 1+0.5+0 = 1.5
rule5  = ctrl.Rule(cost_norm['low']    & clicks_norm['medium'] & impressions_norm['medium'], ranking['high'])   # score: 1+0.5+0.5 = 2
rule6  = ctrl.Rule(cost_norm['low']    & clicks_norm['medium'] & impressions_norm['high'],   ranking['high'])   # score: 1+0.5+1 = 2.5

# Aturan untuk kombinasi Cost (low), Clicks (high), Impressions (?)
rule7  = ctrl.Rule(cost_norm['low']    & clicks_norm['high']   & impressions_norm['low'],    ranking['high'])   # score: 1+1+0 = 2
rule8  = ctrl.Rule(cost_norm['low']    & clicks_norm['high']   & impressions_norm['medium'], ranking['high'])   # score: 1+1+0.5 = 2.5
rule9  = ctrl.Rule(cost_norm['low']    & clicks_norm['high']   & impressions_norm['high'],   ranking['high'])   # score: 1+1+1 = 3

# Aturan untuk kombinasi Cost (medium), Clicks (low), Impressions (?)
rule10 = ctrl.Rule(cost_norm['medium'] & clicks_norm['low']    & impressions_norm['low'],    ranking['low'])    # score: 0.5+0+0 = 0.5
rule11 = ctrl.Rule(cost_norm['medium'] & clicks_norm['low']    & impressions_norm['medium'], ranking['low'])    # score: 0.5+0+0.5 = 1
rule12 = ctrl.Rule(cost_norm['medium'] & clicks_norm['low']    & impressions_norm['high'],   ranking['medium']) # score: 0.5+0+1 = 1.5

# Aturan untuk kombinasi Cost (medium), Clicks (medium), Impressions (?)
rule13 = ctrl.Rule(cost_norm['medium'] & clicks_norm['medium'] & impressions_norm['low'],    ranking['low'])    # score: 0.5+0.5+0 = 1
rule14 = ctrl.Rule(cost_norm['medium'] & clicks_norm['medium'] & impressions_norm['medium'], ranking['medium']) # score: 0.5+0.5+0.5 = 1.5
rule15 = ctrl.Rule(cost_norm['medium'] & clicks_norm['medium'] & impressions_norm['high'],   ranking['high'])   # score: 0.5+0.5+1 = 2

# Aturan untuk kombinasi Cost (medium), Clicks (high), Impressions (?)
rule16 = ctrl.Rule(cost_norm['medium'] & clicks_norm['high']   & impressions_norm['low'],    ranking['medium']) # score: 0.5+1+0 = 1.5
rule17 = ctrl.Rule(cost_norm['medium'] & clicks_norm['high']   & impressions_norm['medium'], ranking['high'])   # score: 0.5+1+0.5 = 2
rule18 = ctrl.Rule(cost_norm['medium'] & clicks_norm['high']   & impressions_norm['high'],   ranking['high'])   # score: 0.5+1+1 = 2.5

# Aturan untuk kombinasi Cost (high), Clicks (low), Impressions (?)
rule19 = ctrl.Rule(cost_norm['high']   & clicks_norm['low']    & impressions_norm['low'],    ranking['low'])    # score: 0+0+0 = 0
rule20 = ctrl.Rule(cost_norm['high']   & clicks_norm['low']    & impressions_norm['medium'], ranking['low'])    # score: 0+0+0.5 = 0.5
rule21 = ctrl.Rule(cost_norm['high']   & clicks_norm['low']    & impressions_norm['high'],   ranking['low'])    # score: 0+0+1 = 1

# Aturan untuk kombinasi Cost (high), Clicks (medium), Impressions (?)
rule22 = ctrl.Rule(cost_norm['high']   & clicks_norm['medium'] & impressions_norm['low'],    ranking['low'])    # score: 0+0.5+0 = 0.5
rule23 = ctrl.Rule(cost_norm['high']   & clicks_norm['medium'] & impressions_norm['medium'], ranking['low'])    # score: 0+0.5+0.5 = 1
rule24 = ctrl.Rule(cost_norm['high']   & clicks_norm['medium'] & impressions_norm['high'],   ranking['medium']) # score: 0+0.5+1 = 1.5

# Aturan untuk kombinasi Cost (high), Clicks (high), Impressions (?)
rule25 = ctrl.Rule(cost_norm['high']   & clicks_norm['high']   & impressions_norm['low'],    ranking['low'])    # score: 0+1+0 = 1
rule26 = ctrl.Rule(cost_norm['high']   & clicks_norm['high']   & impressions_norm['medium'], ranking['medium']) # score: 0+1+0.5 = 1.5
rule27 = ctrl.Rule(cost_norm['high']   & clicks_norm['high']   & impressions_norm['high'],   ranking['high'])   # score: 0+1+1 = 2

# 7. Gabungkan semua aturan ke dalam satu sistem kontrol fuzzy
rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, 
         rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, 
         rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27]

ranking_ctrl = ctrl.ControlSystem(rules)
ranking_simulation = ctrl.ControlSystemSimulation(ranking_ctrl)

# ====================================
# 5. Fungsi untuk Menghitung Ranking per Baris (dengan data normalisasi)
# ====================================
def compute_ranking(row):
    ranking_simulation.reset()  # Reset simulasi untuk setiap perhitungan

    # Pastikan data normalisasi sudah ada
    if pd.isna(row['Cost_norm']) or pd.isna(row['Clicks_norm']) or pd.isna(row['Impressions_norm']):
        print(f"Data NaN ditemukan pada row {row.name}.")
        return np.nan

    ranking_simulation.input['cost_norm'] = row['Cost_norm']
    ranking_simulation.input['clicks_norm'] = row['Clicks_norm']
    ranking_simulation.input['impressions_norm'] = row['Impressions_norm']

    try:
        ranking_simulation.compute()
    except Exception as e:
        print(f"⚠️ Error pada row {row.name}: {e}")
        return np.nan

    result = ranking_simulation.output.get('ranking', np.nan)
    if np.isnan(result):
        print(f"Row {row.name} menghasilkan NaN, menetapkan default value 0.")
        result = 0
    return result

# Terapkan fungsi compute_ranking ke setiap baris
df['Ranking'] = df.apply(compute_ranking, axis=1)

# Urutkan dataset berdasarkan Ranking (nilai tertinggi di atas)
df_sorted = df.sort_values(by='Ranking', ascending=False)

print("\nHasil pengurutan berdasarkan Ranking (skala 0 sampai 1):")
print(df_sorted[['Ad name', 'Cost', 'Clicks (destination)', 'Impressions', 'Ranking']].head())

# Simpan hasil ke file CSV jika diperlukan
df_sorted.to_csv('sorted_ads_ranking_normalized.csv', index=False)


Data asli:
   Ad name     Cost  Impressions  Clicks (destination)  Rank
0  Iklan A   500.25       150000                  3500     7
1  Iklan B  3200.75      5000000                125000     9
2  Iklan C  1200.50      2000000                 60000     5
3  Iklan D   450.30       100000                  2500     8
4  Iklan E  2500.90      4500000                110000    10

Data setelah normalisasi:
      Cost  Cost_norm  Clicks (destination)  Clicks_norm  Impressions  \
0   500.25   0.018161                  3500     0.008163       150000   
1  3200.75   1.000000                125000     1.000000      5000000   
2  1200.50   0.272755                 60000     0.469388      2000000   
3   450.30   0.000000                  2500     0.000000       100000   
4  2500.90   0.745551                110000     0.877551      4500000   

   Impressions_norm  
0          0.010204  
1          1.000000  
2          0.387755  
3          0.000000  
4          0.897959  

Hasil pengurutan berdasa

In [2]:
df_sorted1 = df_sorted[['Ad name', 'Cost', 'Impressions', 'Clicks (destination)','Ranking']] 
df_sorted1.head(5)

,Ad name,Cost,Impressions,Clicks (destination),Ranking
1,Iklan B,3200.75,5000000,125000,0.833333
8,Iklan I,2900.60,4700000,115000,0.831344
4,Iklan E,2500.90,4500000,110000,0.823826
6,Iklan G,1900.40,3500000,85000,0.805563
9,Iklan J,1350.80,2500000,70000,0.784067
